In [ ]:
! pip install --upgrade codetiming numpy pandas census us mechanicalsoup geopandas pandas_bokeh torch
get_ipython().kernel.do_shutdown(True)
from google.colab import drive
drive.mount()

In [ ]:
repo_path = '/content/drive/MyDrive/gerrymandering/2022-10/voting_predictor'
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload
from model import *
torch.manual_seed(0)
B = torch.FloatTensor([[1,1,1,0,0,0],[0,0,0,1,1,1,]]).T.to(device)

class VotingPredictor(torch.nn.Module):
    def __init__(self, input_size, layer_size, activation, election='all'):
        self.layer_size = listify(layer_size)
        self.activation  = listify(activation)
        self.election    = election
        self.rmse_train  = []
        self.rmse_test   = []
        if len(self.activation) - len(self.layer_size) == 1:
            self.layer_size.append(6)
        assert len(self.layer_size) == len(self.activation), 'layer_size and activation must have same length'
        super().__init__()
        L = []
        p = input_size
        for q, f in zip(self.layer_size, self.activation):
            L.append(torch.nn.Linear(p, q))
            L.append(f())
            p = q
        self.nn = torch.nn.Sequential(*L)
        
    def forward(self, W, X):
        prop = self.nn(X)
        pred = ((prop * W) @ B).squeeze()
        return pred

    def main(self, W, X, Y):
        test_mask = (X.reset_index()['election'] == self.election).values
        train_mask = ~test_mask
        if train_mask.all():
            test_mask = train_mask
        W_test  = torch.FloatTensor(W[test_mask ].values).to(device)
        X_test  = torch.FloatTensor(X[test_mask ].values).to(device)
        Y_test  = torch.FloatTensor(Y[test_mask ].values).to(device)
        W_train = torch.FloatTensor(W[train_mask].values).to(device)
        X_train = torch.FloatTensor(X[train_mask].values).to(device)
        Y_train = torch.FloatTensor(Y[train_mask].values).to(device)

        loss_fcn = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters())
        for k in range(1000):
            optimizer.zero_grad()
            loss = loss_fcn(self(W_train, X_train), Y_train)
            loss.backward()
            optimizer.step()
            self.rmse_train.append(np.sqrt(loss_fcn(self(W_train, X_train), Y_train).item()))
            self.rmse_test .append(np.sqrt(loss_fcn(self(W_test , X_test ), Y_test ).item()))
            r = self.rmse_test[-100:]
            if len(r) >= 100 and np.var(r) / np.mean(r) < 0.0001:
                break
        votes_pred = self(W_test, X_test).detach().cpu().numpy().sum(axis=0).round().astype(int)
        votes_true = Y_test.detach().cpu().numpy().sum(axis=0).round().astype(int)
        votes_err  = votes_pred - votes_true
        pct_pred = 100.0 * votes_pred / votes_pred.sum()
        pct_true = 100.0 * votes_true / votes_true.sum()
        pct_err  = pct_pred - pct_true
        self.results = prep(pd.DataFrame(
            data   =[[self.election,  votes_pred[0],  votes_true[0],  votes_err[0],  votes_pred[1],  votes_true[1],  votes_err[1],  pct_pred[0],  pct_pred[1],  pct_true[0],  pct_true[1],  pct_err[0]]],
            columns= ['election'   , 'votes_pred_d', 'votes_true_d', 'votes_err_d', 'votes_pred_r', 'votes_true_r', 'votes_err_r', 'pct_pred_d', 'pct_pred_r', 'pct_true_d', 'pct_true_r', 'pct_err']))

feat = [
    'dist_to_border', # 'aland', 'polsby_popper',
    # 'white_tot_pop'       , 'hisp_tot_pop'       , 'other_tot_pop'       ,
    # 'white_vap_pop'       , 'hisp_vap_pop'       , 'other_vap_pop'       ,
    'white_vap_density'   , 'hisp_vap_density'   , 'other_vap_density'   ,
    'white_vap_poverty'   , 'hisp_vap_poverty'   , 'other_vap_poverty'   ,
    'white_vap_elderly'   , 'hisp_vap_elderly'   , 'other_vap_elderly'   ,
    'white_vap_highschool', 'hisp_vap_highschool', 'other_vap_highschool',
    'white_vap_homeowner' , 'hisp_vap_homeowner' , 'other_vap_homeowner' ,
    'hisp_vap_spanish_at_home_english_well',
]
targ = ['d', 'r']
weig = ['white_vap_pop', 'hisp_vap_pop', 'other_vap_pop']
elections = ['2020_general_President', '2016_general_President', '2018_general_USSen', '2020_general_USSen']

df = pd.concat([features(*elec.split('_')) for elec in elections])[feat+targ+weig].sample(frac=1)
W = df[weig].astype(float)
X = df[feat].astype(float)
Y = df[targ].astype(float)
W = W.join(W, lsuffix='_d', rsuffix='_r')
X = (X - X.min()) / (X.max() - X.min())
param_grid = list(cartesian({
    'layer_size': range(1, 10, 1),
    'activation': [
        [torch.nn.ReLU, torch.nn.Sigmoid],
        [torch.nn.ReLU, torch.nn.Tanh],
    ],
}))

mkdir(MODEL_PATH, overwrite=True)
results = []
models = []
for k, kwargs in enumerate(param_grid):
    print(k, kwargs)
    E = []
    for election in elections+['all']:
        model = VotingPredictor(election=election, input_size=X.shape[1], **kwargs).to(device)
        model.main(W, X, Y)
        E.append(model)
    res = pd.concat([e.results for e in E], ignore_index=True)
    res.index = 0*res.index + k
    model.results = res
    models.append(model)
    torch.save(model, MODEL_PATH / f'model{str(k).rjust(3,"0")}.pt')

results = pd.concat([model.results for model in models])
summary = (results
    .assign(m = lambda x: x['election'].isin(elections))
    .assign(e = lambda x: x['pct_err'].abs())
    .assign(a = lambda x: x['m'] * x['e'])
    .assign(b = lambda x:~x['m'] * x['e'])
    .groupby(level=0).agg(
        rmse = ('a', lambda x: (x**2).mean()**(1/2)),
        max  = ('e', 'max'),
        all  = ('b', 'max'),
        ct   = ('m', 'sum'),
    )
)
summary.insert(0, 'activation' , [[f.__name__ for f in model.activation] for model in models])
summary.insert(0, 'layer_size', [model.layer_size for model in models])


# summary = results.query('election in @elections').groupby(level=0).agg(
#     rmse = ('pct_err', lambda x: (x**2).mean()**(1/2)),
#     max  = ('pct_err', lambda x: x.abs().max()),
#     ct   = ('pct_err', 'count'),
# )
# summary.insert(0, 'activation' , [[f.__name__ for f in model.activation] for model in models])
# summary.insert(0, 'layer_size', [model.layer_size for model in models])

results.to_csv(MODEL_PATH / 'results.csv')
summary.to_csv(MODEL_PATH / 'summary.csv')
display(results.round(2))
display(summary.sort_values('rmse').round(2))

In [ ]:
w = (results
    .assign(m = lambda x: x['election'].isin(elections))
    .assign(e = lambda x: x['pct_err'].abs())
    .assign(a = lambda x: x['m'] * x['e'])
    .assign(b = lambda x:~x['m'] * x['e'])
    .groupby(level=0).agg(
        rmse = ('a', lambda x: (x**2).mean()**(1/2)),
        max  = ('e', 'max'),
        all  = ('b', 'max'),
        ct   = ('m', 'sum'),
    )
)
w.insert(0, 'activation' , [[f.__name__ for f in model.activation] for model in models])
# w.assign(all=w['election'].isin(elections))
# w['a'] = w['election'].notin(elections)
# w['b'] = (w['pct_err'] * w['a']).abs()
# w['c'] = (w['pct_err'] * ~w['a']).abs()
w.head(8)
# summary = results.groupby(level=0).agg(
#     rmse = ('pct_err', lambda x: (x**2).mean()**(1/2)),
#     max  = ('pct_err', lambda x: x.abs().max()),
#     ct   = ('pct_err', 'count'),
# )


In [ ]:
summary.sort_values('rmse').round(2)

In [ ]:
summary['a']

In [ ]:
# display(summary)
# hs = [p['hidden_size'] for p in param_grid]
hs = [model.hidden_size for model in models]
ac = [[f.__name__ for f in model.activation] for model in models]
# ac = [[f.__name__ for f in p['activation']] for p in param_grid]
hs
summary = summary.drop(columns=['hidden_size', 'activation'])
summary.insert(0, 'activation', ac)
summary.insert(0, 'hidden_size', hs)

summary
# a = param_grid[0]['activation'][0]
# display(a)
# a.__name__

In [ ]:
## old
mount_path = '/content/drive'
repo_path = f'{mount_path}/MyDrive/gerrymandering/2022-10/voting_predictor'
from google.colab import drive
drive.mount(mount_path)
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload
from model import *
%cd {repo_path}
%load_ext google.colab.data_table

%load_ext autoreload
%autoreload
from model import *

class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size, activation):
        super(Feedforward, self).__init__()
        assert len(hidden_size) == len(activation), 'hidden_size and activation must have same length'
        L = []
        p = input_size
        for q, f in zip(hidden_size, activation):
            L.append(torch.nn.Linear(p, q))
            L.append(f())
            p = q
        self.nn = torch.nn.Sequential(*L)
        self.rmse_train = []
        self.rmse_test  = []

    def forward(self, X, W):
        prop = self.nn(X)
        # print(self.prop.shape, W.shape, B.shape)
        pred = ((prop * W) @ B).squeeze()
        # print(self.pred.shape)
        return pred

    def get_rmse(self):
        self.rmse_train.append(np.sqrt(loss_fcn(self(X_train, W_train), Y_train).item()))
        self.rmse_test .append(np.sqrt(loss_fcn(self(X_test , W_test ), Y_test ).item()))


feat = [
    # 'aland',
    # 'polsby_popper',
    'dist_to_border',
    # 'white_tot_pop'       , 'hisp_tot_pop'       , 'other_tot_pop'       ,
    # 'white_vap_pop'       , 'hisp_vap_pop'       , 'other_vap_pop'       ,
    'white_vap_density'   , 'hisp_vap_density'   , 'other_vap_density'   ,
    'white_vap_poverty'   , 'hisp_vap_poverty'   , 'other_vap_poverty'   ,
    'white_vap_elderly'   , 'hisp_vap_elderly'   , 'other_vap_elderly'   ,
    'white_vap_highschool', 'hisp_vap_highschool', 'other_vap_highschool',
    'white_vap_homeowner' , 'hisp_vap_homeowner' , 'other_vap_homeowner' ,
    'hisp_vap_spanish_at_home_english_well',
]
targ = ['d', 'r']
weig = ['white_vap_pop', 'hisp_vap_pop', 'other_vap_pop']

elections = ['2020_general_President', '2016_general_President', '2018_general_USSen', '2020_general_USSen']
df = pd.concat([features(*elec.split('_')) for elec in elections])[feat+targ+weig].sample(frac=1)
W = df[weig].astype(float)
W = W.join(W, lsuffix='_d', rsuffix='_r')
Y = df[targ].astype(float)
X = df[feat].astype(float)
X = (X - X.min()) / (X.max() - X.min())
B = torch.FloatTensor([[1,1,1,0,0,0],[0,0,0,1,1,1,]]).T.to(device)

def voting_predictor(election, activation, hidden=(50, 6)):
    assert hidden[-1]==6, f'hidden must end with 6'
    W_test  = torch.FloatTensor(W.query('election == @election').values).to(device)
    X_test  = torch.FloatTensor(X.query('election == @election').values).to(device)
    Y_test  = torch.FloatTensor(Y.query('election == @election').values).to(device)
    W_train = torch.FloatTensor(W.query('election != @election').values).to(device)
    X_train = torch.FloatTensor(X.query('election != @election').values).to(device)
    Y_train = torch.FloatTensor(Y.query('election != @election').values).to(device)

    model = Feedforward(
        input_size = X_train.shape[1],
        hidden_size = hidden,
        activation = activation,
    ).to(device)
    loss_fcn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    steps = 10000
    for k in range(steps):
        optimizer.zero_grad()  # Clear gradient
        loss = loss_fcn(model(X_train, W_train), Y_train) # Compute train loss
        loss.backward()  # Backward propagation
        optimizer.step()  # Learn
        model.get_rmse()
        r = model.rmse_test[-100:]
        if len(r) >= lag and np.var(r) / np.mean(r) < 0.0001:
            break

    model.election = election
    model.cnt_pred = model(X_test, W_test).detach().cpu().numpy().sum(axis=0).round().astype(int)
    model.cnt_true = Y_test.detach().cpu().numpy().sum(axis=0).round().astype(int)
    model.cnt_err  = model.cnt_pred - model.cnt_true
    model.pct_pred = 100.0 * model.cnt_pred / model.cnt_pred.sum()
    model.pct_true = 100.0 * model.cnt_true / model.cnt_true.sum()
    model.pct_err  = model.pct_pred - model.pct_true
    return model
    # result = {
    #     'cnt_pred':cnt_pred, 'cnt_true':cnt_true, 'cnt_err':cnt_err,
    #     'pct_pred':pct_pred, 'pct_true':pct_true, 'pct_err':pct_err, }


    # print(elec, Y_pred, Y_true, Y_err, result_true.round(2), result_pred.round(2), result_err.round(2))
    
    # err = model(X_test, W_test) - Y_test
    # err_pct = (err.sum(axis=0) / Y_test.sum(axis=0)) * 100
    # print(elec, err_pct.detach().cpu().numpy().round(2))

res = dict()
for activation in [[torch.nn.ReLU, torch.nn.Sigmoid]]:
    for election in elections:
        model = voting_predictor(election, activation, hidden=(50, 6))
        res[election] = model
        print(model.election)
